# Machine Learning - Titanic Problem

Let's first import some standard libraries for data processing and visualization


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
train_data_frame = pd.read_csv("./data/train.csv")
test_data_frame = pd.read_csv("./data/test.csv")

train_data_frame.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Preprocess Data
Let's filter the data to remove any noise

In [3]:
def normalize_column(column_values: np.ndarray):
    return (column_values - np.min(column_values)) / (np.max(column_values) - np.min(column_values))


def pre_process(data_frame: pd.DataFrame) -> pd.DataFrame:
    data_frame = data_frame.drop(['Name', 'Ticket', 'PassengerId'], axis = 1)

    # Replace NaN with "Unknown"
    data_frame.Cabin.fillna('Unknown', inplace = True)
    data_frame.Embarked.fillna('Unknown', inplace = True)

    # Replace missing values with median
    cols = ['SibSp', 'Parch', 'Fare', 'Age']
    for col in cols:
        data_frame[col].fillna(data_frame[col].median(), inplace = True)

    # Normalize the age column
    data_frame['Age'] = normalize_column(data_frame['Age'].values)

    # One-hot encoding categorical values
    data_frame = pd.get_dummies(data_frame, columns=["Sex"], prefix="Sex")
    data_frame = pd.get_dummies(data_frame, columns=["Embarked"], prefix="Embarked")

    return data_frame


In [4]:
train_data_frame = pre_process(train_data_frame)
test_data_frame = pre_process(test_data_frame)

train_data_frame.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_Unknown
0,0,3,0.271174,1,0,7.2500,Unknown,False,True,False,False,True,False
1,1,1,0.472229,1,0,71.2833,C85,True,False,True,False,False,False
2,1,3,0.321438,0,0,7.9250,Unknown,True,False,False,False,True,False
3,1,1,0.434531,1,0,53.1000,C123,True,False,False,False,True,False
4,0,3,0.434531,0,0,8.0500,Unknown,False,True,False,False,True,False
5,0,3,0.346569,0,0,8.4583,Unknown,False,True,False,True,False,False
6,0,1,0.673285,0,0,51.8625,E46,False,True,False,False,True,False
7,0,3,0.019854,3,1,21.0750,Unknown,False,True,False,False,True,False
8,1,3,0.334004,0,2,11.1333,Unknown,True,False,False,False,True,False
9,1,2,0.170646,1,0,30.0708,Unknown,True,False,True,False,False,False


Let's look at the data through some visualizations and see if there's anything immediately obvious.

In [ ]:
fig, ax = plt.subplots(figsize=(20,20)) 
sns.heatmap(train_data_frame.corr(), cmap = 'YlGnBu', annot=True, ax=ax)

Based on the heatmap above, it appears that the most clear correlation between a person's chance of survival is based upon their gender and their fare. From their we can see what constitutes the majority of those respective categories as well.